# Vast DB set up using API

## Overview 

This example uses the Vast VMS API via the [vastpy](https://github.com/vast-data/vastpy) library.

This automation is based on the manual instructions available [here](https://vast-data.github.io/data-platform-field-docs/vast_database/overview/getting_started.html).

This example has been tested on: **5.1.0**

## Instructions

First install the vastpy library

In [ ]:
pip install vastpy

Next create a function `base_setup()` that configures a Vast Database.  The function creates:

- vippool
- user
- user S3 key
- view
- database
- database schema

In [ ]:
from vastpy import VASTClient

def base_setup(address, username, password, config):
    client = VASTClient(user=username, password=password, address=address)

    database_owner = config['user']['name']

    try:
        client.vippools.post(**config['vippool'])
        print(f"VIP pool created: {config['vippool']}")
    except Exception as e:
        print(e)
        raise e

    try:
        client.users.post(**config['user'])
        print(f"User created: {config['user']}")
    except Exception as e:
        print(e)
        raise e

    user = client.users.get(name=database_owner)[0]
    user_id = user['id']

    try:
        response = client.users[user_id].access_keys.post(id=user_id)
        access_key = response['access_key']
        secret_key = response['secret_key']
        print(f"Key Generated for {database_owner}")
    except Exception as e:
        print(e)
        raise e

    try:
        response = client.views.post(
            path=config['viewpath']['path'],
            protocols=config['viewpath']['protocols'],
            create_dir=config['viewpath']['create_dir'],
            bucket=config['viewpath']['bucket'],
            bucket_owner=config['viewpath']['bucket_owner'],
            policy_id=config['viewpath']['policy_id']
        )
        print(f"View Created: {config['viewpath']['path']}")
    except Exception as e:
        print(e)
        raise e

    try:
        response = client.schemas.post(
            name=config['database']['schema'],
            database_name=config['database']['name']
        )
        print("Schema Created")
    except Exception as e:
        print(e)
        raise e

    print(access_key, secret_key)

Ensure the config below matches your environment and requirements

In [ ]:
config = {
    'vippool': {
        'start_ip': '11.0.0.2',
        'end_ip': '11.0.0.3',
        'subnet_cidr': 24
    },
    'user': {
        'local': True,
        'name': 'demo-owner',
        'uid': 555,
        'allow_create_bucket': True,
        's3_superuser': True
    },
    'viewpath': {
        'path': '/demo_path',
        'protocols': ['S3', 'DATABASE'],
        'create_dir': True,
        'bucket': 'demo-db',
        'bucket_owner': 'demo-owner', # must match user.name, above
        'policy_id': 3 
    },
    'database': {
        'name': 'demo-db',  # must match viewpath.bucket, above
        'schema': 'demo-schema'
    }
}

Now call base_setup() with the config details.

In [ ]:

base_setup(
    address='VMS_HOST_OR_IP',
    username='admin',
    password='123456',
    config=config
    )

**DANGER:** The function `clean_env()`, below, will remove the following configuration from your cluster:

- ALL database schemas
- ALL views
- ALL vippools
- ALL users

In [ ]:
def clean_env(address, username, password):
    client = VASTClient(user=username, password=password, address=address)
    for schema in client.schemas.get():
        print(f"Deleting schema {schema['name']} in database {schema['database_name']}")
        try:
            tables = client.tables.get(database_name=schema['database_name'], schema_name=schema['name'])['results']
            for table in tables:
                print(f"Deleting Table {table['name']} in schema {table['schema_name']} in database {table['database_name']}")
                client.tables.delete(database_name=table['database_name'], schema_name=schema['database_name'], name=f"`{table['name']}`")
            print("Tables deleted")
        except Exception as e:
            print(e)
            raise e
        client.schemas.delete(database_name=schema['database_name'], name=schema['name'])
    print("Schemas deleted")

    for view in client.views.get():
        # don't delete the default view
        if view['path'] != '/':
            print(f"Deleting view {view['path']}")
            client.views[view['id']].delete()
    print("Views deleted")

    for pool in client.vippools.get():
        print(f"Deleting vippool {pool['id']}")
        client.vippools[pool['id']].delete()
    print("Vippools deleted")
    
    for user in client.users.get():
        print(f"Deleting user {user['name']}")
        client.users[user['id']].delete()
    print("Users deleted")
